<a href="https://colab.research.google.com/github/uskhbs/Machine_learning/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# 훈련세트와 테스트 세트로 나눈다
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

# 훈련세트와 검증 세트를 만든다 20%
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [2]:
# 모델생성 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [3]:
# 교차 검증
# 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다.
# cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행한다. 
from sklearn.model_selection import cross_validate 
scores = cross_validate(dt, train_input, train_target)
print(scores)

# fit_time : 모델을 훈련하는 시간
# score_time : 검증하는 시간
# 교차검증의 최종점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

import numpy as np
print(np.mean(scores['test_score']))

{'fit_time': array([0.01055026, 0.0119319 , 0.00998974, 0.01000452, 0.01026011]), 'score_time': array([0.0011282 , 0.00113106, 0.0011301 , 0.00088167, 0.00113082]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [4]:
# 분할기 - 교차 검증을 할 때 훈련세트를 섞음

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

# 훈런세트를 섞은 후 10-폴드 교차 검증을 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


In [5]:
# 그리드 서치(Grid Search)
# GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행

from sklearn.model_selection import GridSearchCV

# 기본매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾는다
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# GridSearchCV의 cv 매개변수 기본값은 5
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

# 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어있다.
print(gs.best_params_)

# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성에 'mean_test_score'키에 저장되어있다.
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [6]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001) ,
          'max_depth':range(5, 20, 1),
          'min_samples_split':range(2, 100, 10)
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

# 최상의 매개변수 조합
print(gs.best_params_)

# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [9]:
# 랜덤서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포객체를 전달
from scipy.stats import uniform, randint
rgen = randint(0,10) # 정수값
rgen.rvs(10)

np.unique(rgen.rvs(1000), return_counts=True)

ugen = uniform(0,1) # 실수값
ugen.rvs(10)

array([0.95383609, 0.87552688, 0.03595182, 0.14742633, 0.57209298,
       0.4162029 , 0.71801135, 0.58226874, 0.97060895, 0.00587285])

In [12]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20,50),
    'min_samples_split': randint(2,25),
    'min_samples_leaf': randint(1,25),
}

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42) # n_iter : 샘플링 횟수
gs.fit(train_input, train_target)

# 최적의 매개변수 조합출력
print(gs.best_params_)

# 최고의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

# 최적의 모델 (이미 전체 훈련세트로 훈련되어 best_estimator_ 속에 저장되어있다.)
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
